In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, precision_recall_curve, auc
import matplotlib.pyplot as plt

# 加载数据
data = pd.read_csv("../Data/creditcard.csv")

# 划分数据集
X = data.drop("Class", axis=1)  # 特征
y = data["Class"]  # 目标变量

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 进行 SMOTE 和 TomekLinks 混合采样
smote_tomek = SMOTETomek(sampling_strategy='auto', random_state=42, n_jobs=-1)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)

# 输出每个类别的样本数
print("Class distribution before SMOTE and TomekLinks Sampling:")
print(y_train.value_counts())

print("\nClass distribution after SMOTE and TomekLinks Sampling:")
print(pd.Series(y_resampled).value_counts())

# 可视化类别分布
plt.figure(figsize=(8, 6))
plt.bar(pd.Series(y_resampled).value_counts().index, pd.Series(y_resampled).value_counts().values, color=['blue', 'orange'])
plt.title('Class Distribution after SMOTE and TomekLinks Sampling')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks([0, 1], ['Non-Fraud (0)', 'Fraud (1)'])
plt.show()

# 选择逻辑回归分类器，并设置参数
lr_classifier = LogisticRegression(random_state=42)

# 训练分类器
lr_classifier.fit(X_resampled, y_resampled)

# 预测概率
y_proba_lr = lr_classifier.predict_proba(X_test)[:, 1]

# 计算AUPRC
precision, recall, _ = precision_recall_curve(y_test, y_proba_lr)
auprc_lr = auc(recall, precision)

# 打印AUPRC
print("\nAUPRC (Logistic Regression with SMOTE and TomekLinks Sampling):", auprc_lr)

# 绘制PR曲线
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'AUPRC (Logistic Regression with SMOTE and TomekLinks Sampling) = {auprc_lr:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Logistic Regression with SMOTE and TomekLinks Sampling)')
plt.legend()
plt.show()
